If the smart contracts = 100, smart contracts are missing. 

In [3]:
# load libraries 

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import ast





#chrome_options = Options()
#chrome_options.add_argument("--headless")

# 1. Get dApps

In [220]:
# this function scrapes all dApp names, the category of the dApp, and a link to the dApp subpage
def getdApps(protocol) -> pd.DataFrame:

    i = 1
    url = f'https://dappradar.com/rankings/protocol/{protocol}/{i}'

    CHROMEDRIVERPATH = r"C:\Users\DanielObermeier\OneDrive\Dokumente\PhD\02 - Forschung\00 - Projects\00 Ethereum Data\05 NBER dApp data\dAppRadar Scraper\chromedriver.exe"
    driver = webdriver.Chrome(CHROMEDRIVERPATH)  
    driver.maximize_window()

    # navigate to the ural
    resp = driver.get(url)


    # find the navigation bar at the end of the dApp table
    navigation_bar = driver.find_element_by_class_name("sc-fEOsli.saiYi")

    # find last page 
    last_page = int(navigation_bar.find_elements_by_tag_name('a')[-1].text)


    dApp_list = []


    for j in range(1, last_page): 
        
        # navigate to next dApp ranking page
        url_i = f'https://dappradar.com/rankings/protocol/{protocol}/{j}'
        resp_i = driver.get(url_i)
        
        # wait until page has loaded
        time.sleep(10)

        # find rows of dApp rankings
        rows = driver.find_elements_by_class_name("sc-eKszNL.dYKdGj")

        # iterate trough rows
        for el in rows: 
            temp_list = []
            name = el.find_element_by_class_name('sc-dmWEhm.iUkspK').text
            link = str(el.find_element_by_tag_name('a').get_attribute("href"))
            category = el.find_element_by_class_name('sc-olbas.jowZDT').text

            temp_list.extend([name, link, category])
            dApp_list.append(temp_list)
 
    
    driver.close()

    return pd.DataFrame(dApp_list, columns = ["name", "link", "category"])



In [ ]:
# call the function with all protocols 

avalanch_dApps = getdApps("avalanche")
avalanch_dApps = avalanch_dApps[avalanch_dApps['link'].str.contains("https://dappradar.com/")==True] # filter out adds

bsc_dApps = getdApps("binance-smart-chain")
bsc_dApps = bsc_dApps[bsc_dApps['link'].str.contains("https://dappradar.com/")==True] # filter out adds

solana_dApps = getdApps("solana")
solana_dApps = solana_dApps[solana_dApps['link'].str.contains("https://dappradar.com/")==True] # filter out adds

In [219]:
# save dApps 
avalanch_dApps.to_csv("./Data/avalanchdApps_checkpoint.csv")
solana_dApps.to_csv("./Data/solana_dApps_checkpoint.csv")
bsc_dApps.to_csv("./Data/bsc_dApps_checkpoint.csv")

In [36]:
# load from checkpoint
solana_dApps = pd.read_csv("./Data/solana_dApps_checkpoint.csv", index_col=0)

In [ ]:
avalanch_dApps = pd.read_csv("./Data/avalanchdApps_checkpoint.csv", index_col=0)

In [20]:
bsc_dApps = pd.read_csv("./Data/bsc_dApps_checkpoint.csv", index_col=0)

## 2. Get dApp details

In [13]:
def initWebdriver():
    CHROMEDRIVERPATH = r"C:\Users\DanielObermeier\OneDrive\Dokumente\PhD\02 - Forschung\00 - Projects\00 Ethereum Data\05 NBER dApp data\dAppRadar Scraper\chromedriver.exe"
    driver = webdriver.Chrome(CHROMEDRIVERPATH)  
    driver.maximize_window()

    return driver

def closeWebdriver(driver):
    driver.close()
    print("Driver has been closed")

In [37]:
def getdAppData(url, driver) -> dict:

    resp = driver.get(url)

    # init dict to be returned
    variables = {}

    time.sleep(4)
    # find social links
    try:
        socials = driver.find_element_by_class_name("article-page__social").find_elements_by_tag_name('a')
        socialLinks_list = [[social.get_attribute("href") for social in socials]]
        variables["socials"] = socialLinks_list
    except:
        variables["socials"] = []


    # find stats
    stats_containers = driver.find_elements_by_class_name("article-page__stats-container.article-page__stats-container-enabled")
    if len(stats_containers) != 0:
        for stats_container in stats_containers:
            variables[stats_container.text.splitlines()[0].capitalize()] = stats_container.text.splitlines()[1]
    else:
        print(url, "was not able to find stats")
        return variables


    # find smart contracts 

    # scroll down to button
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight/5);")

    time.sleep(1.5)

    # find button 
    try:
        button = driver.find_element_by_class_name("sc-ifmBRf.cbIygx")

        # scroll down to the button
        actions = ActionChains(driver)
        actions.move_to_element(button)
        actions.perform()

    except:
        print(url, "was not able to find button")
        return variables

    # click button
    try:
        button.click()
    except:
        print(url, "was not able to click button")
        return variables

    time.sleep(3)

    # switch to the window
    driver.switch_to.active_element

    # find smart contracts
    smart_contracts_table = driver.find_element_by_class_name("sc-bRrTXx.gaRFrv").find_elements_by_tag_name('a')

    smartContract_list = []

    for smart_contract in smart_contracts_table: 
        smartContract_list.append(smart_contract.text)


    variables["smart_contracts"] = smartContract_list

        
    return variables


In [ ]:
# test function
#url = "https://dappradar.com/binance-smart-chain/games/gameta"
#url = "https://dappradar.com/thundercore/defi/tt-mining"

driver = initWebdriver()


# test multiple dApps
test2 = getdAppData(url, driver)


closeWebdriver(driver)

## 3. Add dApp data to dApps

#### Solana

In [38]:
# Solana
# init new webdriver window
driver = initWebdriver()

dAppData_list = []

for link in solana_dApps.link:
    dAppData_list.append(getdAppData(link, driver))

solana_dApps["data"] = dAppData_list

# close webdriver
closeWebdriver(driver)

# save results as csv file
solana_dApps.to_csv("./Data/solana_dApps.csv")

https://dappradar.com/solana/defi/solend was not able to find stats
https://dappradar.com/solana/defi/larix was not able to find stats
https://dappradar.com/solana/defi/marinade-finance was not able to find stats
https://dappradar.com/solana/defi/sunny-aggregator was not able to find stats
https://dappradar.com/solana/defi/parrot was not able to find stats
https://dappradar.com/solana/collectibles/solanamonkeybusiness was not able to find button
https://dappradar.com/solana/defi/quarry was not able to find stats
https://dappradar.com/solana/defi/serum was not able to find stats
Driver has been closed


In [34]:
# solana dApps with errors

https://dappradar.com/solana/defi/solend was not able to find stats
https://dappradar.com/solana/defi/larix was not able to find stats
https://dappradar.com/solana/defi/marinade-finance was not able to find stats
https://dappradar.com/solana/defi/sunny-aggregator was not able to find stats
https://dappradar.com/solana/defi/parrot was not able to find stats
https://dappradar.com/solana/collectibles/solanamonkeybusiness was not able to find button
https://dappradar.com/solana/defi/quarry was not able to find stats
https://dappradar.com/solana/defi/serum was not able to find stats

#### Avalanch

In [14]:
driver = initWebdriver()

In [15]:
# Avalanch
# init new webdriver window
#driver = initWebdriver()

dAppData_list = []

for link in avalanch_dApps.link:
    dAppData_list.append(getdAppData(link, driver))

avalanch_dApps["data"] = dAppData_list

# close webdriver
closeWebdriver(driver)



https://dappradar.com/avalanche/gambling/casino-poker-lucky-wheel-keno-bingo-heads-or-tails was not able to find stats
https://dappradar.com/avalanche/collectibles/hatchypocket was not able to find stats
https://dappradar.com/avalanche/games/metaderby was not able to find stats
Driver has been closed


In [ ]:
# avalanch dApps with errors

https://dappradar.com/avalanche/gambling/casino-poker-lucky-wheel-keno-bingo-heads-or-tails was not able to find stats
https://dappradar.com/avalanche/collectibles/hatchypocket was not able to find stats
https://dappradar.com/avalanche/games/metaderby was not able to find stats

In [27]:
# save results as csv file
avalanch_dApps.to_csv("./Data/avalanch_dApps.csv")

In [28]:
solana_dApps

,name,link,category
1,Rarible,https://dappradar.com/solana/marketplaces/rarible,Marketplaces
2,Magic Eden,https://dappradar.com/solana/marketplaces/magi...,Marketplaces
3,Gameta,https://dappradar.com/solana/games/gameta,Games
4,MeanFi,https://dappradar.com/solana/defi/meanfi,DeFi
5,Raydium,https://dappradar.com/solana/defi/raydium,DeFi
...,...,...,...
99,Sentoki NFTs,https://dappradar.com/solana/collectibles/sent...,Collectibles
100,Sentre Protocol,https://dappradar.com/solana/defi/sentre-protocol,DeFi
101,Serum,https://dappradar.com/solana/defi/serum,DeFi
102,Skulls.WTF,https://dappradar.com/solana/collectibles/skul...,Collectibles


#### Binance smart chain

In [23]:
# Avalanch
# init new webdriver window
driver = initWebdriver()

dAppData_list = []

for link in bsc_dApps.link:
    dAppData_list.append(getdAppData(link, driver))

bsc_dApps["data"] = dAppData_list

# close webdriver
closeWebdriver(driver)



https://dappradar.com/binance-smart-chain/defi/pancakeswap was not able to find button
https://dappradar.com/binance-smart-chain/defi/venus-protocol was not able to find stats
https://dappradar.com/binance-smart-chain/games/alien-worlds was not able to click button
https://dappradar.com/binance-smart-chain/other/token-batchsender was not able to find stats
https://dappradar.com/binance-smart-chain/defi/beta-pulsar was not able to find stats
https://dappradar.com/binance-smart-chain/collectibles/bwc-garden-tool-set was not able to find stats
https://dappradar.com/binance-smart-chain/marketplaces/crypto-ultraman-nft was not able to find stats
https://dappradar.com/binance-smart-chain/defi/interlace-network was not able to find stats
https://dappradar.com/binance-smart-chain/defi/jetfuel-finance was not able to find stats
https://dappradar.com/binance-smart-chain/defi/kine-protocol was not able to find stats
https://dappradar.com/binance-smart-chain/defi/rabbit-finance was not able to fin

In [ ]:
# binance dApps with errors 

https://dappradar.com/binance-smart-chain/defi/pancakeswap was not able to find button
https://dappradar.com/binance-smart-chain/defi/venus-protocol was not able to find stats
https://dappradar.com/binance-smart-chain/games/alien-worlds was not able to click button
https://dappradar.com/binance-smart-chain/other/token-batchsender was not able to find stats
https://dappradar.com/binance-smart-chain/defi/beta-pulsar was not able to find stats
https://dappradar.com/binance-smart-chain/collectibles/bwc-garden-tool-set was not able to find stats
https://dappradar.com/binance-smart-chain/marketplaces/crypto-ultraman-nft was not able to find stats
https://dappradar.com/binance-smart-chain/defi/interlace-network was not able to find stats
https://dappradar.com/binance-smart-chain/defi/jetfuel-finance was not able to find stats
https://dappradar.com/binance-smart-chain/defi/kine-protocol was not able to find stats
https://dappradar.com/binance-smart-chain/defi/rabbit-finance was not able to find stats
https://dappradar.com/binance-smart-chain/defi/radar-staking was not able to find stats
https://dappradar.com/binance-smart-chain/defi/rubic was not able to find stats
https://dappradar.com/binance-smart-chain/defi/sakeperp was not able to find stats
https://dappradar.com/binance-smart-chain/defi/sushi was not able to find stats
https://dappradar.com/binance-smart-chain/defi/swamp-finance was not able to find stats
https://dappradar.com/binance-smart-chain/other/tornado-cash was not able to find stats
https://dappradar.com/binance-smart-chain/defi/zfarm was not able to find stats

In [24]:
bsc_dApps.to_csv("./Data/bsc_dApps.csv")

In [25]:
bsc_dApps

,name,link,category,data
1,PancakeSwap,https://dappradar.com/binance-smart-chain/defi...,DeFi,{'socials': [['https://twitter.com/pancakeswap...
2,DRIP,https://dappradar.com/binance-smart-chain/high...,High-Risk,{'socials': [['https://twitter.com/DRIPcommuni...
3,Gameta,https://dappradar.com/binance-smart-chain/game...,Games,{'socials': [['https://twitter.com/Gameta_Offi...
4,X World Games,https://dappradar.com/binance-smart-chain/game...,Games,"{'socials': [['https://twitter.com/xwg_games',..."
5,Era7: Game of Truth,https://dappradar.com/binance-smart-chain/game...,Games,{'socials': [['https://www.facebook.com/groups...
...,...,...,...,...
4363,Zombie Farm,https://dappradar.com/binance-smart-chain/defi...,DeFi,{'socials': [['https://twitter.com/ZombieFarm_...
4364,Zomfi,https://dappradar.com/binance-smart-chain/game...,Games,{'socials': [['https://www.facebook.com/Zomfit...
4365,Zonbal Infinity,https://dappradar.com/binance-smart-chain/coll...,Collectibles,{'socials': [['https://twitter.com/zonbalInfin...
4366,Zonogon,https://dappradar.com/binance-smart-chain/game...,Games,"{'socials': [['https://t.me/zonogon']], 'Users..."


#### Workbench

In [98]:


url = "https://dappradar.com/binance-smart-chain/games/gameta"

In [99]:
driver = initWebdriver()

driver.get(url)

In [155]:
variables = {}


# find social links
socials = driver.find_element_by_class_name("article-page__social").find_elements_by_tag_name('a')
socialLinks_list = [[social.get_attribute("href") for social in socials]]
variables["socials"] = socialLinks_list

In [208]:

# find stats
stats_containers = driver.find_elements_by_class_name("article-page__stats-container.article-page__stats-container-enabled")
for stats_container in stats_containers:
    variables[stats_container.text.splitlines()[0]] = stats_container.text.splitlines()[1]

In [133]:
# find smart contracts 

# scroll down to button
driver.execute_script("window.scrollTo(0,document.body.scrollHeight/5);")

time.sleep(2)

# find button 
button = driver.find_element_by_class_name("sc-ifmBRf.cbIygx")

# click button
button.click()

time.sleep(3)

# switch to the window
driver.switch_to.active_element

# find smart contracts
smart_contracts_table = driver.find_element_by_class_name("sc-bRrTXx.gaRFrv").find_elements_by_tag_name('a')

smartContract_list = []

for smart_contract in smart_contracts_table: 
    smartContract_list.append(smart_contract.text)


variables["smart_contracts"] = smartContract_list


#### Write function

In [ ]:
# get avalanche dApps

avalanch_dApps = getdApps("avalanche")
avalanch_dApps["data"] = [0]*len(avalanch_dApps)

dAppData_list = []


for link in avalanch_dApps.link:
    dAppData_list.append(getdAppData(link))

avalanch_dApps["data"] = dAppData_list

avalanch_dApps.to_csv("./Data/avalanchdApps.csv")

In [72]:
# test function

url = 'https://dappradar.com/avalanche/gambling/casino-poker-lucky-wheel-keno-bingo-heads-or-tails'
driver = initWebdriver()

test2 = getdAppData(url, driver)

IndexError: list index out of range

In [215]:
[0]*len(test)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [70]:
fantomdApps = getdApps("fantom")
fantomdApps["data"] = [0]*len(fantomdApps)

In [75]:
dAppData_list = []

In [ ]:
for link in fantomdApps.link:
    dAppData_list.append(getdAppData(link))

In [85]:
fantomdApps.to_csv("./Data/fantomdApps.csv")

In [82]:
polygondApps = getdApps("polygon")
polygondApps["data"] = [0]*len(polygondApps)

/var/folders/31/03qj75b50_sfxqwdc04734600000gn/T/ipykernel_94699/2150958678.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


In [84]:
dAppData_list = []

In [132]:
for link in polygondApps.link[773:]:
    dAppData_list.append(getdAppData(link))

/var/folders/31/03qj75b50_sfxqwdc04734600000gn/T/ipykernel_94699/3336691562.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [133]:
polygondApps["data"] = dAppData_list

In [135]:
polygondApps.to_csv("./Data/polygondApps.csv")

In [137]:
optimismdApps = getdApps("optimism")
optimismdApps["data"] = [0]*len(optimismdApps)

/var/folders/31/03qj75b50_sfxqwdc04734600000gn/T/ipykernel_94699/2150958678.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


In [138]:
dAppData_list = []

In [141]:
for link in optimismdApps.link:
    dAppData_list.append(getdAppData(link))

/var/folders/31/03qj75b50_sfxqwdc04734600000gn/T/ipykernel_94699/3336691562.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


KeyboardInterrupt: 

In [143]:
optimismdApps["data"] = dAppData_list
optimismdApps.to_csv("./Data/optimismdApps.csv")

In [144]:
ethereumdApps = getdApps("ethereum")
ethereumdApps["data"] = [0]*len(ethereumdApps)

/var/folders/31/03qj75b50_sfxqwdc04734600000gn/T/ipykernel_94699/2150958678.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)


In [145]:
dAppData_list = []

In [171]:
for link in ethereumdApps.link[3100:]:
    dAppData_list.append(getdAppData(link))

/var/folders/31/03qj75b50_sfxqwdc04734600000gn/T/ipykernel_94699/3820452127.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [172]:
dAppData_list.insert(3099, "nothin")

In [173]:
ethereumdApps["data"] = dAppData_list

In [199]:
ethereumdApps["smartContracts"] = list(map(lambda x: x.get("smart_contracts"), ethereumdApps.data))

In [201]:
ethereumdApps.to_csv("./Data/ethereumdApps.csv")

In [205]:
ethereumdApps.iloc[3099]

name                                                  TAZOR FINANCE
link              https://dappradar.com/ethereum/defi/tazor-finance
category                                                       DeFi
data                                              {'data': 'empty'}
smartContracts                                                 None
Name: 3099, dtype: object

In [42]:
polygon = pd.read_csv("./Data/polygondApps.csv", index_col=0)
polygon["smartContracts"] = list(map(lambda x: ast.literal_eval(x).get("smart_contracts"), polygon.data))

In [43]:
contractCounts = []
for i in range(len(polygon)):
    contractCounts.append(len(polygon.smartContracts.iloc[i]))

In [44]:
polygon[pd.Series(contractCounts) == 100]

,name,link,category,data,smartContracts
0,QuickSwap,https://dappradar.com/polygon/exchanges/quickswap,Exchanges,{'socials': ['https://twitter.com/QuickswapDEX...,"[0x7295304b10740ba8e037826787d3e9386fd99925, 0..."
23,ApeSwap,https://dappradar.com/polygon/defi/apeswap,DeFi,"{'socials': ['https://twitter.com/ape_swap', '...","[0xc0788a3ad43d79aa53b09c2eacc313a787d1d607, 0..."
52,Beefy Finance,https://dappradar.com/polygon/defi/beefy-finance,DeFi,{'socials': ['https://twitter.com/beefyfinance...,"[0x9649e6E5c689f21BC27B47CE4177f7c5f7281E20, 0..."
97,Autofarm,https://dappradar.com/polygon/defi/autofarm,DeFi,{'socials': ['https://twitter.com/autofarmnetw...,"[0x53baE3Ff063eB3C7B78610dc314eE2067e036d17, 0..."
122,XDAO,https://dappradar.com/polygon/defi/xdao,DeFi,"{'socials': ['https://twitter.com/xdaoapp', 'h...","[0xCA49EcF7e7bb9bBc9D1d295384663F6BA5c0e366, 0..."
139,KogeFarm,https://dappradar.com/polygon/defi/kogefarm,DeFi,"{'socials': ['https://twitter.com/kogecoin', '...","[0x2bb510a979fe3f277636A94B86FA304c6631581C, 0..."
152,ETHA Lend,https://dappradar.com/polygon/defi/etha-lend,DeFi,"{'socials': ['https://twitter.com/ethalend', '...","[0x33D2C07558181bcDd9edBf39C5128BcB43A52Ea7, 0..."


# Old functions

## 1. Get dApps 

In [ ]:
# Function that gets all dApps 

def getdApps(protocol) -> pd.DataFrame:

    i = 1
    url = f'https://dappradar.com/rankings/protocol/{protocol}/{i}'

    # init Webdriver
    CHROMEDRIVERPATH = r"C:\Users\DanielObermeier\OneDrive\Dokumente\PhD\02 - Forschung\00 - Projects\00 Ethereum Data\05 NBER dApp data\dAppRadar Scraper\chromedriver.exe"
    driver = webdriver.Chrome(CHROMEDRIVERPATH)  
    driver.maximize_window()

    resp = driver.get(url)
    time.sleep(10)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    table = soup.find("div", {"class": "sc-IAann exiOPk rankings-table".split(" ")})
    table_elements = list(filter(lambda x: not x.has_attr("rel"), table.find_all("a")))
    
    last_entry = max([int(s) for s in soup.select("div.sc-kDDrLX.egagrd")[0].get_text().split() if s.isdigit()])
    
    element_list = []
    
    for j in range(1, last_entry + 1): 
        url_i = f'https://dappradar.com/rankings/protocol/{protocol}/{j}'
        resp_i = driver.get(url_i)
        time.sleep(10)
        
        soup_j = BeautifulSoup(driver.page_source, "html.parser")
    
        table = soup_j.find("div", {"class": "sc-IAann exiOPk rankings-table".split(" ")})
        table_elements = list(filter(lambda x: not x.has_attr("rel"), table.find_all("a")))

        for el in table_elements:
            temp_list = []
            name = el.get("title")
            link = "https://dappradar.com" + str(el.get("href"))
            if name == None:
                continue
            category = el.select("div.sc-izEbhJ.gYDeNV.rankings-column.rankings-column__category")[0].get_text()
            temp_list.extend([name, link, category])
            element_list.append(temp_list)
    
    driver.close()
            
    return pd.DataFrame(element_list, columns = ["name", "link", "category"])

## 2. Get dApp data

In [ ]:
def getdAppData(url) -> dict:
    # get the social links and recommended apps
    driver = webdriver.Chrome(ChromeDriverManager().install())

    resp = driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    
    list_info = soup.select("div.article-page__stats-container")
    variables = {}
    
    socials = soup.find_all("a", {"class": "article-page__social-icon"})
    socialLinks_list = [social["href"] for social in socials]
    variables["socials"] = socialLinks_list
    
    for el in list_info:
        variable = el.select("div.sc-iiNSSV.bSwoss")[0].get_text()
        value = el.select("div.css-otg2q8")[0].get_text() 
        variables[variable] = value
        
    button = driver.find_element(By.CSS_SELECTOR, "div.sc-diYIgC.laNies")
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight/6);")

    time.sleep(2)
    button.click()
    
    time.sleep(2)
    
    driver.switch_to.active_element
    soup = BeautifulSoup(driver.page_source, "html.parser")
    time.sleep(5)
    
    smart_contracts = soup.select("ul.sc-jStShS.jDAcLg")[0].find_all("li")
    smartContract_list = []
    for contract in smart_contracts:
        element = contract.find("a").get_text()
        smartContract_list.append(element)
        
    variables["smart_contracts"] = smartContract_list
    
    driver.close()
    
    return variables

In [ ]:
def getdAppData(url, driver) -> dict:

    resp = driver.get(url)

    # init dict to be returned
    variables = {}

    time.sleep(1)
    # find social links
    socials = driver.find_element_by_class_name("article-page__social").find_elements_by_tag_name('a')
    socialLinks_list = [[social.get_attribute("href") for social in socials]]
    variables["socials"] = socialLinks_list


    # find stats
    stats_containers = driver.find_elements_by_class_name("article-page__stats-container.article-page__stats-container-enabled")
    if len(stats_containers) != 0:
        for stats_container in stats_containers:
            variables[stats_container.text.splitlines()[0].capitalize()] = stats_container.text.splitlines()[1]
    else:
        print(url, "was not able to find stats")
        return variables


    # find smart contracts 

    # scroll down to button
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight/5);")

    time.sleep(2)

    # find button 
    try:
        button = driver.find_element_by_class_name("sc-ifmBRf.cbIygx")

    except:
        print(url, "was not able to find button")
        return variables

    # click button
    button.click()

    time.sleep(3)

    # switch to the window
    driver.switch_to.active_element

    # find smart contracts
    smart_contracts_table = driver.find_element_by_class_name("sc-bRrTXx.gaRFrv").find_elements_by_tag_name('a')

    smartContract_list = []

    for smart_contract in smart_contracts_table: 
        smartContract_list.append(smart_contract.text)


    variables["smart_contracts"] = smartContract_list

        
    return variables
